In [2]:
import requests, json
from collections import Counter
import pandas as pd
import numpy as np

# Get all WGS metagenomes

**fields to filter on:**

*metagenome filters*

{"field":"study_type","type":8, "value":["Metagenomics","Whole Genome Sequencing", "Whole Genome Sequencing ", "Other"]},

{"field":"sequencing_method", "type":7, "value":["illumina","Illumina"]},

{"field":"library_source", "type":8, "value":["metagenomic"]},

{"field":"library_strategy", "type": 8, "value":["WGS", "WGA", "OTHER"]},

{"field":"library_reads_sequenced_maxrun", "type":4, "value":1000000}]}'}

In [2]:
rules_items = [{"field":"study_type","type":8, "value":["Metagenomics","Whole Genome Sequencing", "Whole Genome Sequencing ", "Other"]},
               {"field":"sequencing_method", "type":7, "value":["illumina","Illumina"]},
               {"field":"library_source", "type":8, "value":["metagenomic"]},
               {"field":"library_strategy", "type": 8, "value":["WGS", "WGA", "OTHER"]},
               {"field":"library_reads_sequenced_maxrun", "type":4, "value":1000000}]
               
rules = {"rules":rules_items}
post_data = {'filter_params':json.dumps(rules)}


In [3]:
import requests, json
post = requests.post('https://api.metaseek.cloud/datasets/search/ids', data=post_data)
result = json.loads(post.text) #the parsed result returned from the API.

#this result will be a dictionary with a list of the matching dataset MetaSeek UIDs, a count of the number of matching datasets, and the filter parameters that were used
print(result.keys())
matching_datasets = result['matching_dataset_ids']


dict_keys(['filter_params', 'count_matching_datasets', 'matching_dataset_ids'])


In [8]:
print(len(matching_datasets))

107709


In [4]:
#loop through batches of 1000 IDs and query for full metadata for each batch
def get_batches(uid_list, batch_size):
    starts = list(range(0,len(uid_list),batch_size))
    ends = list(range(batch_size,len(uid_list),batch_size))
    ends.append(len(uid_list))
    batches = [list(a) for a in zip(starts, ends)]
    return batches

batch_indexes = get_batches(matching_datasets, batch_size=1000)

In [5]:
metagenomes = pd.DataFrame()
for batch in batch_indexes:
  batch_id_list = matching_datasets[batch[0]:batch[1]]
  post_data = {'metaseek_ids':str(batch_id_list)}
  print('getting metadata for batch ', batch)
  post_metadata = requests.post('https://api.metaseek.cloud/datasets/metadatafromids', data=post_data)
  result = json.loads(post_metadata.text)
  metagenomes = metagenomes.append(result['datasets'], ignore_index=True)

getting metadata for batch  [0, 1000]
getting metadata for batch  [1000, 2000]
getting metadata for batch  [2000, 3000]
getting metadata for batch  [3000, 4000]
getting metadata for batch  [4000, 5000]
getting metadata for batch  [5000, 6000]
getting metadata for batch  [6000, 7000]
getting metadata for batch  [7000, 8000]
getting metadata for batch  [8000, 9000]
getting metadata for batch  [9000, 10000]
getting metadata for batch  [10000, 11000]
getting metadata for batch  [11000, 12000]
getting metadata for batch  [12000, 13000]
getting metadata for batch  [13000, 14000]
getting metadata for batch  [14000, 15000]
getting metadata for batch  [15000, 16000]
getting metadata for batch  [16000, 17000]
getting metadata for batch  [17000, 18000]
getting metadata for batch  [18000, 19000]
getting metadata for batch  [19000, 20000]
getting metadata for batch  [20000, 21000]
getting metadata for batch  [21000, 22000]
getting metadata for batch  [22000, 23000]
getting metadata for batch  [2300

In [9]:
metagenomes.head()

,age,altitude,assembly,avg_read_length_maxrun,baseA_count_maxrun,baseC_count_maxrun,baseG_count_maxrun,baseN_count_maxrun,baseT_count_maxrun,biomaterial_provider,...,study_type_other,submission_id,subspecific_genetic_lineage,target_gene,target_subfragment,taxon_common_name,taxon_scientific_name,tissue,total_num_bases_maxrun,uri
0,None,None,None,101.0000,328906237,253135919,258309914,1540558,322212162,None,...,None,SRA142047,None,None,None,None,human metagenome,G_DNA_Stool,1164104790,/dataset/471
1,None,None,None,251.0000,2530564082,2365540151,2381416521,1688418,2487627528,None,...,None,ERA319098,None,None,None,None,metagenome,None,9766836700,/dataset/472
2,None,None,None,118.2430,251876372,424083904,429011348,63249,245816809,None,...,None,SRA536700,Sample 372,None,None,None,indoor metagenome,None,1350851682,/dataset/474
3,None,None,None,114.2160,270678592,319116231,322950324,54335,265850402,None,...,None,SRA536700,Sample 374,None,None,None,indoor metagenome,None,1178649884,/dataset/476
4,None,None,None,98.2786,1698704767,1357826371,1352551982,34006,1698900055,None,...,None,ERA675994,None,None,None,Gut metagenome,gut metagenome,None,6108017181,/dataset/501


In [24]:
print(np.sum(metagenomes['library_reads_sequenced_maxrun'])/1e12)
np.sum(metagenomes['download_size_maxrun'])/1e12

2.284282536712


287.986866871226

if I used ALL of the metagenomes, I would :

-- have 2.28 TRILLION reads to input to the model

-- have to go through 288TB of raw data for every batch! (and that's just the .sra files!!)

maybe I can start with 1/100th of that amount and see how it goes... that will be about 20 billion inputs and 2.9TB to download

In [26]:
metagenomes.to_csv('AllWGSMetagenomes_111519.csv',index=False)

In [3]:
metagenomes = pd.read_csv("AllWGSMetagenomes_111519.csv")
metagenomes.head()

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,2,4,6,7,17,20,25,26,27,29,42,63,68,73,91,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,age,altitude,assembly,avg_read_length_maxrun,baseA_count_maxrun,baseC_count_maxrun,baseG_count_maxrun,baseN_count_maxrun,baseT_count_maxrun,biomaterial_provider,...,study_type_other,submission_id,subspecific_genetic_lineage,target_gene,target_subfragment,taxon_common_name,taxon_scientific_name,tissue,total_num_bases_maxrun,uri
0,NaN,NaN,NaN,101.0000,328906237,2.531359e+08,258309914,1540558,3.222122e+08,NaN,...,NaN,SRA142047,NaN,NaN,NaN,NaN,human metagenome,G_DNA_Stool,1.164105e+09,/dataset/471
1,NaN,NaN,NaN,251.0000,2530564082,2.365540e+09,2381416521,1688418,2.487628e+09,NaN,...,NaN,ERA319098,NaN,NaN,NaN,NaN,metagenome,NaN,9.766837e+09,/dataset/472
2,NaN,NaN,NaN,118.2430,251876372,4.240839e+08,429011348,63249,2.458168e+08,NaN,...,NaN,SRA536700,Sample 372,NaN,NaN,NaN,indoor metagenome,NaN,1.350852e+09,/dataset/474
3,NaN,NaN,NaN,114.2160,270678592,3.191162e+08,322950324,54335,2.658504e+08,NaN,...,NaN,SRA536700,Sample 374,NaN,NaN,NaN,indoor metagenome,NaN,1.178650e+09,/dataset/476
4,NaN,NaN,NaN,98.2786,1698704767,1.357826e+09,1352551982,34006,1.698900e+09,NaN,...,NaN,ERA675994,NaN,NaN,NaN,Gut metagenome,gut metagenome,NaN,6.108017e+09,/dataset/501


In [4]:
np.histogram(metagenomes['library_reads_sequenced_maxrun'].dropna(), bins=1000)

(array([16903,  8562,  9277,  5238,  5654,  5548,  4712,  3778,  3204,
         3131,  2874,  2649,  2164,  2170,  1993,  1772,  1648,  1466,
         1376,  1384,  1301,  1240,  1136,  1133,   881,   907,   816,
          878,   751,   682,   654,   541,   538,   476,   460,   425,
          390,   379,   348,   286,   268,   257,   221,   221,   209,
          208,   181,   152,   148,   138,  2027,   121,   125,   111,
          125,    95,    96,   109,   100,   106,    98,    89,   104,
           92,    80,    81,    72,    72,    73,    77,    56,    91,
           46,    63,    75,    62,    45,    47,    53,    48,    61,
           55,    45,    55,    72,    65,    30,    31,    44,    37,
           29,    38,    45,    40,    38,    29,    26,    33,    39,
           34,    34,    37,    22,    49,    36,    25,    41,    33,
           36,    32,    41,    42,    39,    34,    34,    31,    34,
           32,    38,    37,    40,    31,    49,    27,    50,    43,
      

# Get a reasonable subset of training data from the 100k metagenomes

need download size to be below my 20TB hard disk limit - and remember, the .sra download size is maybe 5-10x smaller than the final .fastq size.

~evenly distributed across env_package, biome, etc.

In [5]:
Counter(metagenomes['metaseek_investigation_type'])

Counter({'metagenome': 104863,
         'mimarks-culture': 130,
         'bacteria_archaea': 142,
         'mimarks-survey': 4135,
         'eukaryote': 230,
         'virus': 106,
         nan: 1})

In [6]:
new = metagenomes[metagenomes['metaseek_investigation_type']=='metagenome']

In [7]:
new.shape

(104863, 98)

In [8]:
Counter(new['library_construction_method'])

Counter({'paired': 93288, 'single': 11575})

In [78]:
paired = new[new['library_construction_method']=='paired']
len(paired)

93288

In [79]:
Counter(paired['metaseek_env_package']).most_common()

[(nan, 68104),
 ('human-gut', 14823),
 ('water', 3566),
 ('host-associated', 2759),
 ('soil', 1244),
 ('human-skin', 1129),
 ('sediment', 387),
 ('wastewater/sludge', 386),
 ('plant-associated', 247),
 ('miscellaneous', 235),
 ('human-associated', 147),
 ('microbial mat/biofilm', 101),
 ('built environment', 86),
 ('human-oral', 74)]

In [9]:
Counter(new['metaseek_env_package']).most_common()

[(nan, 73871),
 ('human-gut', 17458),
 ('host-associated', 5356),
 ('water', 3665),
 ('soil', 1334),
 ('human-skin', 1129),
 ('sediment', 451),
 ('wastewater/sludge', 411),
 ('plant-associated', 364),
 ('miscellaneous', 307),
 ('human-associated', 214),
 ('human-oral', 115),
 ('microbial mat/biofilm', 102),
 ('built environment', 86)]

In [10]:
#I'm going to group all the human stuff together so we now have ten total categories
human = ['human-gut','human-skin','human-associated','human-oral']
other = ['water','host-associated','soil','sediment','wastewater/sludge','plant-associated','miscellaneous','microbial mat/biofilm','built environment']

In [11]:
len(new[new['metaseek_env_package']=='water'])

3665

In [12]:
len(new[new['metaseek_env_package'].isin(human)])

18916

In [73]:
#randomly select up to 100 metagenomes from each category
even = pd.DataFrame()
for cat in other:
    subset = new[new['metaseek_env_package']==cat]
    if len(subset)>=100:
        toadd = subset.sample(100,replace=False, random_state=1000)
    else:
        toadd = subset
    even = even.append(toadd)
subseth = new[new['metaseek_env_package'].isin(human)]
toaddh = subseth.sample(100, replace=False, random_state=1000)
even = even.append(toaddh)

In [80]:
#randomly select up to 100 metagenomes from each category
even_paired = pd.DataFrame()
for cat in other:
    subset = paired[paired['metaseek_env_package']==cat]
    if len(subset)>=100:
        toadd = subset.sample(100,replace=False, random_state=1000)
    else:
        toadd = subset
    even_paired = even_paired.append(toadd)
subseth = paired[paired['metaseek_env_package'].isin(human)]
toaddh = subseth.sample(100, replace=False, random_state=1000)
even_paired = even_paired.append(toaddh)

In [81]:
len(even_paired)

986

In [99]:
subset = even_paired[even_paired['metaseek_env_package']=='host-associated']
subset.sample(1,replace=False)[['run_ids_maxrun', 'study_title']]

,run_ids_maxrun,study_title
86033,ERR2241972,"As part of the EFFORT project, we sampled fece..."


In [104]:
Counter(subset['taxon_common_name'])
subset[subset['taxon_common_name']=='domestic cat'][['run_ids_maxrun', 'study_title']]

,run_ids_maxrun,study_title
40769,ERR878249,A longitudinal study of the feline faecal micr...
27919,ERR878219,A longitudinal study of the feline faecal micr...


In [74]:
len(even)

986

In [41]:
even.head()

,age,altitude,assembly,avg_read_length_maxrun,baseA_count_maxrun,baseC_count_maxrun,baseG_count_maxrun,baseN_count_maxrun,baseT_count_maxrun,biomaterial_provider,...,study_type_other,submission_id,subspecific_genetic_lineage,target_gene,target_subfragment,taxon_common_name,taxon_scientific_name,tissue,total_num_bases_maxrun,uri
8779,None,None,None,251.0,117008482,136079007,133412647,0,116232784,None,...,None,SRA274315,None,None,None,None,freshwater metagenome,None,502732920,/dataset/647855
20895,None,None,None,230.0,380756028,262065707,257678960,8189,381403878,None,...,None,ERA413491,None,None,None,None,marine metagenome,None,1281912762,/dataset/1554820
29034,None,None,None,97.0,10704029068,6727683875,6662886930,20622902,10741983542,None,...,None,ERA357871,None,None,None,None,marine metagenome,None,34857206317,/dataset/2181948
48626,None,None,None,146.0,4139249822,3706151668,3626880611,58773,4094571895,None,...,None,ERA990579,None,None,None,None,marine metagenome,None,15566912769,/dataset/3213949
9383,None,None,None,108.0,242081065,190221364,190514339,242927,240940305,None,...,None,ERA392462,None,None,None,None,metagenome,None,864000000,/dataset/689853


In [42]:
even['run_ids_maxrun'][0:20]

8779     SRR2089207
20895     ERR770995
29034     ERR599082
48626    ERR2098376
9383      ERR694137
48483    ERR1987991
75227    ERR2604105
9433      ERR868363
18327     ERR598947
20498     ERR868350
32896    SRR1029019
3722     ERR1662521
28116     SRR931753
14867     ERR594372
13445    SRR1230756
23441    ERR1662337
2580     ERR1726622
2007     ERR1726530
19848     ERR599042
85353    ERR2841088
Name: run_ids_maxrun, dtype: object

In [46]:
impt_fields = [
    'sample_title',
    'study_title',
    'metaseek_env_package',
    'taxon_scientific_name',
    'env_biome',
    'env_feature',
    'env_material',
    'metaseek_investigation_type',
    'meta_latitude',
    'meta_longitude',
    'geo_loc_name',
    'expt_link'
]
even[impt_fields]

,sample_title,study_title,metaseek_env_package,taxon_scientific_name,env_biome,env_feature,env_material,metaseek_investigation_type,meta_latitude,meta_longitude,geo_loc_name,expt_link
8779,MIMS Environmental/Metagenome sample from fres...,watershed biome Raw sequence reads,water,freshwater metagenome,watershed biome,protected,fresh water,metagenome,49.0000,-123.00000,Canada: Southwestern British Columbia,https://www.ncbi.nlm.nih.gov/sra/1588960
20895,59; OSD52_2014-06-21_1m_NPL022,Ocean Sampling Day (OSD) 2014: AUTHORITY-RAW a...,water,marine metagenome,marine biome,surface water,seawater,metagenome,NaN,NaN,None,https://www.ncbi.nlm.nih.gov/sra/1288662
29034,TARA_B100000073; TARA_20100315T1118Z_038_EVENT...,Shotgun Sequencing of Tara Oceans DNA samples ...,water,marine metagenome,marine biome (ENVO:00000447),deep chlorophyll maximum layer (ENVO:xxxxxxxx),"saline water (ENVO:00002010), including plankt...",metagenome,NaN,NaN,None,https://www.ncbi.nlm.nih.gov/sra/986402
48626,EMOSE_N010000424; EMOSE_201705300746Z_DAY1_PUM...,EMOSE (2017) Inter-Comparison of Marine Plankt...,water,marine metagenome,Westerlies Biome,[SRF] surface water layer (ENVO:00010504) with...,"water (ENVO:00002006), including particulate m...",metagenome,NaN,NaN,None,https://www.ncbi.nlm.nih.gov/sra/4402402
9383,FS896_ElGuapo_DNA; Metagenomes and metatranscr...,Metagenomes and metatranscriptomes from the di...,water,metagenome,marine,hydrothermal vent,diffuse fluid,metagenome,45.9266,-129.97900,Pacific Ocean,https://www.ncbi.nlm.nih.gov/sra/1210603
48483,174; GOAIL,"We analyzed 9,933 larvae from 383 samples in t...",water,metagenome,sea water,open water,sea water,metagenome,29.4478,34.96280,Jordan,https://www.ncbi.nlm.nih.gov/sra/4589557
75227,SOLA35,SOLA (NW Mediterranean) surface seawater metag...,water,seawater metagenome,None,None,None,metagenome,NaN,NaN,Mediterranean Sea,https://www.ncbi.nlm.nih.gov/sra/5859821
9433,TARA_N000002037; TARA_20110804T1833Z_124_EVENT...,Shotgun Sequencing of Tara Oceans DNA samples ...,water,marine metagenome,marine biome (ENVO:00000447),surface water layer (ENVO:00002042),"particulate matter, including plankton (ENVO:x...",metagenome,NaN,NaN,None,https://www.ncbi.nlm.nih.gov/sra/1476189
18327,TARA_B100000470; TARA_20100913T1303Z_068_Combi...,Shotgun Sequencing of Tara Oceans DNA samples ...,water,marine metagenome,marine biome (ENVO:00000447),mesopelagic zone (ENVO:00000213),"saline water (ENVO:00002010), including plankt...",metagenome,NaN,NaN,None,https://www.ncbi.nlm.nih.gov/sra/986573
20498,TARA_N000000419; TARA_20100629T1500Z_058_EVENT...,Shotgun Sequencing of Tara Oceans DNA samples ...,water,marine metagenome,marine biome (ENVO:00000447),deep chlorophyll maximum layer (ENVO:xxxxxxxx),"particulate matter, including plankton (ENVO:x...",metagenome,NaN,NaN,None,https://www.ncbi.nlm.nih.gov/sra/1476250


In [47]:
#this is fine for now. Save!
even.to_csv('Metagenomes1000_EvenEnvPackages.csv',index=False)

In [38]:
#randomly select up to 100 metagenomes from each category
small = pd.DataFrame()
for cat in other:
    subset = new[new['metaseek_env_package']==cat]
    if len(subset)>=1:
        toadd = subset.sample(1,replace=False, random_state=99)
    else:
        toadd = subset
    small = small.append(toadd)
subseth = new[new['metaseek_env_package'].isin(human)]
toaddh = subseth.sample(1, replace=False, random_state=99)
small = small.append(toaddh)

In [39]:
small.head()
print(np.mean(small['avg_read_length_maxrun']))
print(np.mean(small['library_reads_sequenced_maxrun'])/1e6)

116.9
25.812524600000003


In [109]:
subseth = new[new['metaseek_env_package'].isin(human)]
subseth.sample(1, replace=False)[['run_ids_maxrun', 'study_title']]

,run_ids_maxrun,study_title
17591,ERR1753690,An obese child finished a 105-days intervenion...


In [60]:
subset = new[new['metaseek_env_package']=='sediment']
subset.sample(1,replace=False)[['run_ids_maxrun', 'study_title']]

,run_ids_maxrun,study_title
27904,ERR257715,Making and breaking DMS by salt marsh microbes...


In [62]:
subset = new[new['metaseek_env_package']=='miscellaneous']
subset.sample(1,replace=False)[['run_ids_maxrun', 'study_title']]

,run_ids_maxrun,study_title
60342,ERR2298555,Bacterial and viral communities within ferment...


In [67]:
subset = new[new['metaseek_env_package']=='microbial mat/biofilm']
subset.sample(1,replace=False)[['run_ids_maxrun', 'study_title']]

,run_ids_maxrun,study_title
30212,ERR1543536,Metagenomic analysis of microbial mat of hot s...


In [68]:
subset = new[new['metaseek_env_package']=='built environment']
subset.sample(1,replace=False)[['run_ids_maxrun', 'study_title']]

,run_ids_maxrun,study_title
18011,ERR1332584,Hospital Metagenomics


In [40]:
small[['library_reads_sequenced_maxrun','metaseek_env_package', 'run_ids_maxrun']]

,library_reads_sequenced_maxrun,metaseek_env_package,run_ids_maxrun
17995,34099068.0,water,SRR5002316
14847,69820263.0,host-associated,SRR925829
9066,48806409.0,soil,ERR671922
52914,2732691.0,sediment,ERR1743390
23717,5217070.0,wastewater/sludge,SRR1702228
2715,23573715.0,plant-associated,SRR1510983
7584,31246537.0,miscellaneous,SRR4048938
32003,14561833.0,microbial mat/biofilm,ERR1855536
21403,26412190.0,built environment,SRR1577780
35294,1655470.0,human-gut,ERR748323


In [69]:
small['run_ids_maxrun']

17995    SRR5002316
14847     SRR925829
9066      ERR671922
52914    ERR1743390
23717    SRR1702228
2715     SRR1510983
7584     SRR4048938
32003    ERR1855536
21403    SRR1577780
35294     ERR748323
Name: run_ids_maxrun, dtype: object

In [70]:
small_runs = [
'SRR5002316',
'SRR925829',
'ERR671922',
'ERR257715',
'SRR1702228',
'SRR1510983',
'ERR2298555',
'ERR1543536',
'ERR1332584',
'ERR748323'
]

In [75]:
np.sum(even['download_size_maxrun'])


3411346244982.0

In [151]:
#randomly select up to 100 metagenomes from each category
ten = pd.DataFrame()
for x in range(0,10):
    #doing it this way so env packages are cycled through in order
    print(x)
    for cat in other:
        subset = new[new['metaseek_env_package']==cat]
        if len(subset)>=1:
            toadd = subset.sample(1,replace=False)
        else:
            toadd = subset
        ten = ten.append(toadd)
    subseth = new[new['metaseek_env_package'].isin(human)]
    toaddh = subseth.sample(1, replace=False)
    ten = ten.append(toaddh)

0
1
2
3
4
5
6
7
8
9


In [152]:
ten['metaseek_env_package']

19493                     water
92916           host-associated
90192                      soil
3172                   sediment
95795         wastewater/sludge
82727          plant-associated
34256             miscellaneous
103967    microbial mat/biofilm
33010         built environment
4847                  human-gut
6047                      water
21030           host-associated
54190                      soil
92216                  sediment
23089         wastewater/sludge
83958          plant-associated
93731             miscellaneous
7171      microbial mat/biofilm
73810         built environment
36141                 human-gut
4125                      water
77887           host-associated
61078                      soil
73545                  sediment
76746         wastewater/sludge
72822          plant-associated
70973             miscellaneous
24116     microbial mat/biofilm
73807         built environment
17049                 human-gut
                  ...          
67457   

In [153]:
print(len(ten[ten['library_reads_sequenced_maxrun']<4000000]))
ten[ten['library_reads_sequenced_maxrun']<4000000][['metaseek_env_package','library_reads_sequenced_maxrun']]

16


,metaseek_env_package,library_reads_sequenced_maxrun
3172,sediment,1975171.0
92216,sediment,1104063.0
73545,sediment,1666623.0
17600,host-associated,1489053.0
82440,plant-associated,2473124.0
21931,human-gut,1575450.0
34209,soil,1354319.0
61252,human-gut,2512512.0
67457,water,2209131.0
94991,host-associated,1018807.0


In [154]:
Counter(ten['library_construction_method'])

Counter({'paired': 85, 'single': 15})

In [156]:
pwd

'/Users/adrienne/Projects/ENIGMA/GetMetaSeekDatasets'

In [155]:
ten['run_ids_maxrun'].to_csv('TrainSmall100_EvenEnv_RunIDS.csv', index=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


## looking closer at the df 

In [48]:
np.mean(even['avg_read_length_maxrun'])

125.6600299188641

In [53]:
np.sum(even['library_reads_sequenced_maxrun'])/1e9

26.26716882

In [50]:
np.mean(even['avg_read_length_maxrun'])*np.sum(even['library_reads_sequenced_maxrun'])/1e12

3.3007332198050543

In [54]:
#26 billion reads; 3.3 trillion nucleotides; 125 avg read length

In [58]:
even['run_ids_maxrun'].to_csv('Metagenomes1000_EvenEnvPackages_RunIDs.csv', index=False)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [57]:
np.where(even.columns=='run_ids_maxrun')

(array([72]),)

In [22]:
runs = pd.read_csv("TrainSmall100_EvenEnv_RunIDS.csv",header=None)
runs.columns = ['run_ids_maxrun']
runs.head()

,run_ids_maxrun
0,ERR1726943
1,ERR2200674
2,ERR1960504
3,SRR4069407
4,ERR3173383


In [47]:
len(runs)
len(set(runs['run_ids_maxrun']))
runs[runs.duplicated()]
runs[runs['run_ids_maxrun']=='SRR925829']
#runs[runs['run_ids_maxrun']=='SRR1577782']

,run_ids_maxrun
41,SRR925829
51,SRR925829
81,SRR925829
91,SRR925829


In [55]:
run_subset = metagenomes[metagenomes['run_ids_maxrun'].isin(runs['run_ids_maxrun'])]
print(len(run_subset))
print(len(run_subset.drop_duplicates()))
run_subset = run_subset.drop_duplicates()
print(len(run_subset))
run_subset.head()
from collections import Counter
Counter(run_subset['metaseek_env_package'])

1993
96
96


Counter({'sediment': 10,
         'water': 10,
         'built environment': 9,
         'human-gut': 9,
         'wastewater/sludge': 10,
         'microbial mat/biofilm': 10,
         'human-skin': 1,
         'host-associated': 7,
         'miscellaneous': 10,
         'soil': 10,
         'plant-associated': 10})

In [56]:
print(len(run_subset))
run_subset.index = run_subset['run_ids_maxrun']
run_subset.head()

96


,age,altitude,assembly,avg_read_length_maxrun,baseA_count_maxrun,baseC_count_maxrun,baseG_count_maxrun,baseN_count_maxrun,baseT_count_maxrun,biomaterial_provider,...,study_type_other,submission_id,subspecific_genetic_lineage,target_gene,target_subfragment,taxon_common_name,taxon_scientific_name,tissue,total_num_bases_maxrun,uri
run_ids_maxrun,,,,,,,,,,,,,,,,,,,,,
SRR4069407,NaN,NaN,NaN,113.0000,109820727,1.135239e+08,113307807,115606,1.096206e+08,NaN,...,NaN,SRA455700,NaN,NaN,NaN,NaN,marine sediment metagenome,NaN,4.463886e+08,/dataset/231812
ERR1474613,NaN,NaN,NaN,122.1590,3536934911,2.383904e+09,2380320821,0,3.526199e+09,NaN,...,NaN,ERA665490,NaN,NaN,NaN,NaN,Achromatium oxaliferum,NaN,1.182736e+10,/dataset/285280
SRR4343439,NaN,NaN,NaN,97.1558,1755872147,1.641800e+09,1648856804,340691,1.762343e+09,NaN,...,NaN,SRA481916,NaN,NaN,NaN,NaN,groundwater metagenome,NaN,6.809213e+09,/dataset/310558
ERR1332600,NaN,NaN,NaN,101.0000,1817642524,1.636167e+09,1476234535,871176,1.917134e+09,NaN,...,NaN,ERA587179,NaN,NaN,NaN,NaN,metagenome,NaN,6.848049e+09,/dataset/347315
ERR209626,NaN,NaN,NaN,68.5843,861254275,7.019861e+08,695705989,20042,8.587082e+08,NaN,...,NaN,ERA179067,NaN,NaN,NaN,NaN,human gut metagenome,NaN,3.117675e+09,/dataset/365915


In [63]:
joined_runs = runs.join(run_subset,on='run_ids_maxrun',lsuffix="1").drop_duplicates()
joined_runs = joined_runs[joined_runs.columns[1:]]

joined_runs = joined_runs[joined_runs['metaseek_env_package']!='human-skin']
print(Counter(joined_runs['metaseek_env_package']))
joined_runs

Counter({'water': 10, 'soil': 10, 'sediment': 10, 'wastewater/sludge': 10, 'plant-associated': 10, 'miscellaneous': 10, 'microbial mat/biofilm': 10, 'built environment': 9, 'human-gut': 9, 'host-associated': 7})


,age,altitude,assembly,avg_read_length_maxrun,baseA_count_maxrun,baseC_count_maxrun,baseG_count_maxrun,baseN_count_maxrun,baseT_count_maxrun,biomaterial_provider,...,study_type_other,submission_id,subspecific_genetic_lineage,target_gene,target_subfragment,taxon_common_name,taxon_scientific_name,tissue,total_num_bases_maxrun,uri
0,NaN,NaN,NaN,97.0000,9936406815,1.114231e+10,11037984069,191833,9.887896e+09,NaN,...,NaN,ERA763626,NaN,NaN,NaN,NaN,marine metagenome,NaN,4.200479e+10,/dataset/1304153
1,NaN,NaN,NaN,93.0000,9406405292,8.301602e+09,8287524401,64590,9.425793e+09,NaN,...,NaN,ERA1138909,NaN,NaN,NaN,NaN,bovine metagenome,NaN,3.542139e+10,/dataset/5690979
2,NaN,NaN,NaN,101.0000,3444474066,5.271387e+09,5303207041,1644344,3.436626e+09,NaN,...,NaN,ERA912527,NaN,NaN,NaN,NaN,soil metagenome,NaN,1.745734e+10,/dataset/5521303
3,NaN,NaN,NaN,113.0000,109820727,1.135239e+08,113307807,115606,1.096206e+08,NaN,...,NaN,SRA455700,NaN,NaN,NaN,NaN,marine sediment metagenome,NaN,4.463886e+08,/dataset/231812
4,NaN,NaN,NaN,251.0000,1398353534,1.252008e+09,1233811816,146370,1.387044e+09,NaN,...,NaN,ERA1745989,NaN,NaN,NaN,NaN,anaerobic digester metagenome,NaN,5.271364e+09,/dataset/5860492
5,NaN,NaN,NaN,101.0000,1174503914,1.108530e+09,1234409039,177014,1.098175e+09,NaN,...,NaN,ERA1614303,NaN,NaN,NaN,NaN,metagenome,NaN,4.615795e+09,/dataset/5055175
6,NaN,NaN,NaN,101.0000,12733097740,1.315828e+10,13193400432,72126767,1.265164e+10,NaN,...,NaN,DRA004030,NaN,NaN,NaN,NaN,activated sludge metagenome,NaN,5.180854e+10,/dataset/2428842
7,NaN,NaN,NaN,147.0000,524872391,4.919386e+08,492473524,0,5.049023e+08,NaN,...,NaN,ERA966087,NaN,NaN,NaN,NaN,Sus scrofa,NaN,2.014187e+09,/dataset/6331978
8,NaN,NaN,NaN,101.0000,1226899244,1.003467e+09,978564199,997908,1.251025e+09,NaN,...,NaN,ERA587179,NaN,NaN,NaN,NaN,metagenome,NaN,4.460954e+09,/dataset/2334070
9,NaN,NaN,NaN,68.5843,861254275,7.019861e+08,695705989,20042,8.587082e+08,NaN,...,NaN,ERA179067,NaN,NaN,NaN,NaN,human gut metagenome,NaN,3.117675e+09,/dataset/365915


In [64]:
joined_runs.to_csv('TrainSmall95_EvenEnv.csv')

In [66]:
print(set(joined_runs['run_ids_maxrun']))

{'ERR1855552', 'ERR2017141', 'SRR925829', 'ERR2239841', 'ERR2145413', 'SRR1298754', 'ERR1190830', 'ERR970605', 'ERR2241851', 'SRR4069407', 'ERR1877921', 'ERR2144884', 'ERR2020015', 'ERR1739691', 'SRR1238204', 'ERR1743388', 'ERR3521978', 'ERR1726943', 'ERR977414', 'ERR598955', 'SRR5164026', 'ERR1743304', 'ERR2699809', 'ERR1135232', 'ERR2239851', 'ERR1960627', 'ERR1855546', 'ERR1358726', 'ERR209703', 'SRR1185414', 'ERR1726572', 'ERR2709750', 'ERR3053395', 'ERR2200674', 'ERR2603191', 'ERR1743283', 'SRR1171647', 'ERR1746303', 'SRR830624', 'ERR712383', 'ERR2239840', 'ERR2298558', 'ERR209626', 'ERR2022395', 'ERR1474613', 'SRR1577782', 'ERR694158', 'ERR1332606', 'ERR2699805', 'ERR2765138', 'ERR2969987', 'ERR977422', 'SRR1574704', 'ERR1855538', 'SRR5091465', 'SRR1238205', 'DRR046818', 'ERR1698988', 'ERR1366724', 'ERR1076075', 'ERR1201181', 'SRR2556884', 'ERR599252', 'ERR2215874', 'DRR027592', 'ERR2709726', 'ERR1700691', 'SRR1511001', 'ERR2819892', 'ERR1332600', 'ERR2200504', 'SRR4343439', 'ERR

In [71]:
#there is one run in the validset that is not in joined_runs: SRR3184383
#there is one run in joined_runs that is not in validset: DRR046818
print(metagenomes[metagenomes['run_ids_maxrun']=='SRR3184383']['metaseek_env_package']) #the human skin one I purposefully took out
print(metagenomes[metagenomes['run_ids_maxrun']=='DRR046818']['metaseek_env_package']) #probably random result of the split; no big

13647    human-skin
Name: metaseek_env_package, dtype: object
34256    miscellaneous
Name: metaseek_env_package, dtype: object


In [5]:
metagenomes = pd.read_csv("AllWGSMetagenomes_111519.csv")
metagenomes.head()

/Users/adrienne/anaconda3/envs/fastai/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (1,2,4,6,7,17,20,25,26,27,29,42,63,68,73,91,92,93) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,age,altitude,assembly,avg_read_length_maxrun,baseA_count_maxrun,baseC_count_maxrun,baseG_count_maxrun,baseN_count_maxrun,baseT_count_maxrun,biomaterial_provider,...,study_type_other,submission_id,subspecific_genetic_lineage,target_gene,target_subfragment,taxon_common_name,taxon_scientific_name,tissue,total_num_bases_maxrun,uri
0,NaN,NaN,NaN,101.0000,328906237,2.531359e+08,258309914,1540558,3.222122e+08,NaN,...,NaN,SRA142047,NaN,NaN,NaN,NaN,human metagenome,G_DNA_Stool,1.164105e+09,/dataset/471
1,NaN,NaN,NaN,251.0000,2530564082,2.365540e+09,2381416521,1688418,2.487628e+09,NaN,...,NaN,ERA319098,NaN,NaN,NaN,NaN,metagenome,NaN,9.766837e+09,/dataset/472
2,NaN,NaN,NaN,118.2430,251876372,4.240839e+08,429011348,63249,2.458168e+08,NaN,...,NaN,SRA536700,Sample 372,NaN,NaN,NaN,indoor metagenome,NaN,1.350852e+09,/dataset/474
3,NaN,NaN,NaN,114.2160,270678592,3.191162e+08,322950324,54335,2.658504e+08,NaN,...,NaN,SRA536700,Sample 374,NaN,NaN,NaN,indoor metagenome,NaN,1.178650e+09,/dataset/476
4,NaN,NaN,NaN,98.2786,1698704767,1.357826e+09,1352551982,34006,1.698900e+09,NaN,...,NaN,ERA675994,NaN,NaN,NaN,Gut metagenome,gut metagenome,NaN,6.108017e+09,/dataset/501


In [12]:
ten = pd.read_csv('TrainSmall100_EvenEnv_RunIDS.csv',names=['id'])
ten.head()

,id
0,ERR1726943
1,ERR2200674
2,ERR1960504
3,SRR4069407
4,ERR3173383


In [13]:
hundo = metagenomes[metagenomes['run_ids_maxrun'].isin(ten['id'])]

In [17]:
hundo = hundo.drop_duplicates()
len(hundo)

96

In [18]:
Counter(hundo['metaseek_env_package'])

Counter({'sediment': 10,
         'water': 10,
         'built environment': 9,
         'human-gut': 9,
         'wastewater/sludge': 10,
         'microbial mat/biofilm': 10,
         'human-skin': 1,
         'host-associated': 7,
         'miscellaneous': 10,
         'soil': 10,
         'plant-associated': 10})

In [21]:
cut = hundo[['metaseek_env_package','run_ids_maxrun']]
cut.columns = ['metaseek_environmental_package','sra_run_accession']
cut.to_csv('EvenEnv100_metagenomes.csv',index=False)